In [184]:
import re

In [185]:
MAP_FILE_LOCATION = "F:\\Projects\\MicroController\\STM32F446RE\\sotom_v1\\build\\final.map"
MAP_FILE = open(MAP_FILE_LOCATION)
MAP_LINES = MAP_FILE.readlines()
MAP_LINE = ""
SECTION = ''
SUB_SECTION = ''
FILE_NAME = ''
VAR_TYPE=''
PRV_ORIGIN = None
section = []
sub_section = []
variables = []
origins = []
filename = []
vartype = []
lengthVar = []

In [186]:
def isSection(line):
    if len(line) != 0 and line.startswith('.'):
        return True
    else:
        return False

def isSubSection(line):
    if len(line) != 0 and line.startswith(' ') and line.lstrip().startswith('.'):
        return True
    else:
        return False


def isValueSection(line):
    if len(line) != 0 and line.startswith('  ') and (not line.lstrip().startswith('.') or not line.lstrip().startswith('*')):
        return True
    else:
        return False

In [187]:
for MAP_LINE in MAP_LINES:
    
    LINE_SPLIT = MAP_LINE.split()
    if len(LINE_SPLIT) == 0:
        continue
    
    if isSection(MAP_LINE):
        SECTION = LINE_SPLIT[0]
        SUB_SECTION = None
        FILE_NAME = None

    if isSubSection(MAP_LINE):
        SUB_SECTION = LINE_SPLIT[0]
        FILE_NAME = "".join(map(str,LINE_SPLIT[3:]))

    if isValueSection(MAP_LINE):
        VAR_TYPE='address'
        SIZE=0
        if SECTION =='.comment' or SECTION=='.ARM.attributes':
            continue
        origins.append(int(LINE_SPLIT[0],16))
        
        if PRV_ORIGIN != None:
            SIZE = int(LINE_SPLIT[0],16)-PRV_ORIGIN
        
        if LINE_SPLIT[1] == '.':
            if len(LINE_SPLIT)>=4:
                variables.append('align:'+LINE_SPLIT[4])
            else:
                variables.append('unknown')
            FILE_NAME=None
        #found end marker
        elif len(LINE_SPLIT)>3 and LINE_SPLIT[3]=='.':
            variables.append(LINE_SPLIT[1])
            FILE_NAME=None
            VAR_TYPE='value'
        elif len(LINE_SPLIT)>3 and LINE_SPLIT[3]=='LOADADDR':
            variables.append(LINE_SPLIT[1])
            FILE_NAME=None
            VAR_TYPE='loadaddr:'+LINE_SPLIT[4]
        else:
            variables.append(LINE_SPLIT[1])

        if PRV_ORIGIN != None:
            if len(filename)!=0 and filename[-1] == None:
                SIZE = 0
            lengthVar.append(SIZE)
        
        section.append(SECTION)
        sub_section.append(SUB_SECTION)
        filename.append(FILE_NAME)
        vartype.append(VAR_TYPE)
        PRV_ORIGIN = int(LINE_SPLIT[0],16)


In [188]:
with open('data.csv','w') as f:
    lis=[section,sub_section,variables,origins,lengthVar,vartype,filename]
    f.write("section,subsection,variable,origin,size,type,filename\n")
    for x in zip(*lis):
        f.write("{0},{1},{2},{3},{4},{5},{6}\n".format(*x))